In [94]:
# %load Resolve.py
# Resolve.py
"""
We consider a normal order ramified on snc but no secondary ramification. We
blow up repeatedly at nodes recording the log discrepancy and
b-discrepancy until we have all possible negative discrepancy curves.

The exceptional curves should probably be indexed by dyadic fractions, written
as binary numbers, but we will instead clear denominators and work with
integers. 
"""

from fractions import Fraction as FR


# Enter initial ramification data.
# The ramification cover of the curve i is gi copies of zi modulo n
#initial_ram_data = input("Please enter the ramification data tuple n,z1,z2,g1,g2\n")
#n_str, z1_str, z2_str, g1_str, g2_str = initial_ram_data.split(',')
#n = int(n_str.strip())
#z1 = int(z1_str.strip())
#z2 = int(z2_str.strip())
#g1 = int(g1_str.strip())
#g2 = int(g2_str.strip())


#This function computes the order of an element in a cyclic group.
def order(num,modulus):
    ord = 1
    while ord*num %modulus !=0:
        ord +=1
    return ord


#print("Corresponding maximal order has ram data", end=':')
#print(z1 % n, z2 % n, ' modulo ', n, end='')
#f1 = order(z1,n) # these are ramification indices of max order
#f2 = order(z2,n)
#print(' giving ramification indices ', f1, f2,'.')
# Check working by showing ramification of containing maximal order



"""
This is the function that computes log and b-discrepancies of
all exceptional curves in a good log resolution. Here the good log
resolution is one chosen so all negative discrepancy curves are
guaranteed to be found there. The data will be recorded in a dictionary.
"""
def resolve(n,z1,z2,g1,g2):
    f1 = order(z1,n) # these are ramification indices of max order
    f2 = order(z2,n)
    no_blowups = max(g1,g2)*n -1
    # crude bound on number of blowups to ensure good log resolution
    print(f'We need to blowup set of nodes  at most {no_blowups} times.')
    no_curves = 2**no_blowups # number of exc curves actually 2**no_blowups -1
    curves = {0:[Fraction(1,f1*g1)-1,z1] , no_curves:[Fraction(1,f2*g2)-1,z2]}
    # Create dictionary of curve data. Keys are 0 up to no_curves.
    # Current values are lists of [log discrepancy, ram of max order] 
    step = no_curves # Final indices for strict transforms of initial ram 
                     # curves are `step' apart

    
    for i in range(1,no_blowups+1):

 #       print(f'This is the {i}-th blowup. New curves have  log and b- discrepancy and ramification:')
        l = len(curves)-1 # the number of new exceptional curves
       
        oldcurves = set(curves.keys()).difference({0,no_curves})
        for index in oldcurves:
            curves[index] = [curves[index][0], curves[index][1], curves[index][2],curves[index][3]-2]                                                         
        step = int(step/2) # indices of i-th blowup curves have indices 'step' apart
        
        is_good_res = True
        # We'll use this to test if the i-th blowup gives a good resolution

        for j in range(l):
            log_disc = 1 + curves[2*j*step][0] + curves[(2*j+2)*step][0]
            #  Give log discrepancy of j-th new exceptional in i-th blowup
            is_good_res =  is_good_res and (log_disc >=0)
            ram = (curves[2*j*step][1] + curves[(2*j+2)*step][1])%n
            # Gives ramification along j-th new exceptional in i-th blowup
            b_disc = log_disc + 1-Fraction(1,order(ram,n))
            # Compute the b-discrepancy
            curves[(2*j +1)*step] = [log_disc, ram, b_disc,-1]
            # Update curves dictionary with entry for j-th curve in i-th blowup
#            print(log_disc,b_disc, ram, end=';')

        if is_good_res:
            break
        else:
            print(' DONE', end='\n\n')
    indices = sorted(curves)
    curvesList = [ curves[i] for i in indices]
    curvesList[0].extend([0,0])
    curvesList[len(curvesList)-1].extend([0,0])
    print(curvesList)    
    return curvesList
#    print('\n\n Good log resolution has been achieved.')

def toNegativeFractions(sequence):
    return [-Fraction(x,1) for x in sequence]
    
def HJcontinuedFraction(sequence):
#    print(sequence)
    if len(sequence) == 1:
#        print(sequence[0])
        return sequence[0]
    else:
        first = sequence.pop(0)
        return first-1/HJcontinuedFraction(sequence)
    
def cyclicGroup(seq):
    frac = HJcontinuedFraction(toNegativeFractions(seq))
    r = frac.numerator
    b = frac.denominator
    return([r,1,b])

def contract2smooth(curves):
    noCurves=len(curves)
    i = 1
    while i < noCurves:
        if curves[i][2] >= 0 and curves[i][3] == -1:
#            print(i)
            noCurves=len(curves)
            curves[min(i+1,noCurves-2)][3] += 1
            curves[max(i-1,1)][3] += 1
            curves.pop(i)
            print([x[3] for x in curves])        
            i = 1
            noCurves -= 1
        else:
            i += 1
    return curves

def contract2min(curves):
    # scan until you find a b des >= 0
    # then scan until you have b des < 0
    noCurves = len(curves)
    smoothPt = [0]
    for b in range (0,noCurves):
        curves.insert(b*2,smoothPt)
    curves.pop(0) 
    # this is a sequence for data for curves and points
    i = 1
    while i < noCurves:        
        print(curves)
        print(noCurves)
        print(i)
        if curves[2*i][2] >= 0:
            start = i
            j = i+1
            while j < noCurves and curves[2*j][2] >= 0:
                j += 1
            stop = j
            print('start stop are:',start,stop)
            print('to contract are:',[curves[2*x] for x in range(start,stop)])
            cg = cyclicGroup([curves[2*x][3] for x in range(start,stop)])
            print(cg)
            print(curves)
            del curves[2*start-1:2*stop]
            print(curves)
            curves.insert(2*start-1,cg)
            print(curves)
        i += 1    
        noCurves = (len(curves)-1)/2
    return(curves)

#resolve(n,z1,z2,g1,g2)


In [95]:
cv = resolve(5,3,1,1,1)
cvsmth = contract2smooth(cv)
contract2min(cvsmth)

We need to blowup set of nodes  at most 4 times.
 DONE

 DONE

 DONE

[[Fraction(-4, 5), 3, 0, 0], [Fraction(0, 1), 3, Fraction(4, 5), -1], [Fraction(-1, 5), 0, Fraction(-1, 5), -3], [Fraction(2, 5), 2, Fraction(6, 5), -1], [Fraction(-2, 5), 2, Fraction(2, 5), -5], [Fraction(3, 5), 3, Fraction(7, 5), -1], [Fraction(0, 1), 1, Fraction(4, 5), -3], [Fraction(2, 5), 0, Fraction(2, 5), -1], [Fraction(-3, 5), 4, Fraction(1, 5), -7], [Fraction(2, 5), 3, Fraction(6, 5), -1], [Fraction(0, 1), 4, Fraction(4, 5), -3], [Fraction(3, 5), 4, Fraction(7, 5), -1], [Fraction(-2, 5), 0, Fraction(-2, 5), -5], [Fraction(2, 5), 1, Fraction(6, 5), -1], [Fraction(-1, 5), 1, Fraction(3, 5), -3], [Fraction(0, 1), 2, Fraction(4, 5), -1], [Fraction(-4, 5), 1, 0, 0]]
[0, -2, -1, -5, -1, -3, -1, -7, -1, -3, -1, -5, -1, -3, -1, 0]
[0, -1, -4, -1, -3, -1, -7, -1, -3, -1, -5, -1, -3, -1, 0]
[0, -1, -3, -2, -1, -7, -1, -3, -1, -5, -1, -3, -1, 0]
[0, -1, -3, -1, -6, -1, -3, -1, -5, -1, -3, -1, 0]
[0, -1, -2, -5, -1, -3,

[[Fraction(-4, 5), 3, 0, 0],
 [0],
 [Fraction(-1, 5), 0, Fraction(-1, 5), -1],
 [5, 1, 3],
 [Fraction(-2, 5), 0, Fraction(-2, 5), -1],
 [0],
 [Fraction(-4, 5), 1, 0, 0]]

In [72]:
x = HJcontinuedFraction(toNegativeFractions([-2]))
x.numerator
x.denominator


[Fraction(2, 1)]
2


1

In [52]:
contract2smooth(cv)

IndexError: list index out of range

In [41]:
indices = list(range(16))
indices
[i^2 for i in indices]

[2, 3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13]

In [74]:
cv[-1]

[Fraction(-4, 5), 1, 0, 0]

In [75]:
cv

[[Fraction(-4, 5), 3, 0, 0],
 [Fraction(-1, 5), 0, Fraction(-1, 5), -1],
 [Fraction(-2, 5), 2, Fraction(2, 5), -2],
 [Fraction(-3, 5), 4, Fraction(1, 5), -3],
 [Fraction(-2, 5), 0, Fraction(-2, 5), -1],
 [Fraction(-4, 5), 1, 0, 0]]

In [76]:
cv[-1]

[Fraction(-4, 5), 1, 0, 0]